***Importações***

In [60]:
import psycopg2
import pandas as pd
from dotenv import load_dotenv
import os


***Conexão ao Banco de Dados do 1° e do 2°*** 

In [61]:
# congigurar variáveis de ambiente
load_dotenv()
# Conectar ao banco PostgreSQL do 1° ano
try:
    conn_1ano = psycopg2.connect(
        dbname=os.getenv("DBNAME_1ano"),
        user=os.getenv("USER"),
        password=os.getenv("PASSWORD"),
        host=os.getenv("HOST"),
    )
    cur_1ano = conn_1ano.cursor()
except Exception as e:
    print("Erro ao conectar com o banco de dados do 1°ano: ",e)

# Conectar ao banco PostgreSQL do 2° ano
try:
    conn_2ano = psycopg2.connect(
        dbname=os.getenv("DBNAME_2ANO"),
        user=os.getenv("USER"),
        password=os.getenv("PASSWORD"),
        host=os.getenv("HOST"),
    )
    cur_2ano = conn_2ano.cursor()
except Exception as e:
    print("Erro ao conectar com o banco de dados do 2°ano: ",e)

***Teste para Verificação de Conexão ao Banco do 1° ano***

In [62]:
try:
    # Testando a conexão com o banco do 1° ano
    query = "SELECT * FROM plano LIMIT 3;"
    cur_1ano.execute(query)
    rows = cur_1ano.fetchall()

    for row in rows:
        print(row)
except Exception as e:
    print("Erro: ",e)
    

(1, '3 TB', 'Básico', 49.91)


***Teste para Verificação de Conexão ao Banco do 2° ano***

In [63]:
try:
    # Testando a conexão com o banco do 1° ano
    query = "SELECT * FROM planos LIMIT 3;"
    cur_2ano.execute(query)
    rows = cur_2ano.fetchall()

    for row in rows:
        print(row)
except Exception as e:
    print("Erro: ",e)

(1, 'Básico', Decimal('49.90'), '2 TB')
(2, 'Intermediário', Decimal('199.90'), '3 TB')
(3, 'Corporativo', Decimal('299.90'), '4 TB')


***Inicialização de Variáveis com o Nome das Tabelas e seus Campos para Obter Maior Facilidade na Manutenção do Código.***

In [64]:
tabelas_1ano = {"plano": ["id","nome","preco","armazenamento"],
                "condena":["id","nome","tipo"], 
                "empresa":["id","nome","cnpj"]
                }


tabelas_2ano = {"planos": ["id","nome","preco","armazenamento"], 
                "condena":["id","nome","tipo"],
                "empresa":["id","nome"]
                }

***Início do RPA***

In [65]:
# Função que realiza o RPA entre as duas tabelas
def RPA_inter(tabela_1ano, colunas_1ano, tabela_2ano, colunas_2ano, conn_1ano_param, conn_2ano_param):

    cursor_1ano = conn_1ano_param.cursor()
    cursor_2ano = conn_2ano_param.cursor()

    # Transforma a lista em texto separado por vírgulas para colocar na query
    colunas_1ano_str = ", ".join(colunas_1ano)
    colunas_2ano_str = ", ".join(colunas_2ano)
    
    # Monta placeholders (%s, %s, ...) -- vários %s apenas para o número de colunas
    placeholders = ", ".join(["%s"] * len(colunas_2ano))

    # Exclude feito com for para não atualizar o id
    excluded = ", ".join([
        f"{col} = EXCLUDED.{col}" for col in colunas_2ano if col != "id"
    ])

    condicoes = " OR ".join([
        f"{tabela_2ano}.{col} IS DISTINCT FROM EXCLUDED.{col}"
        for col in colunas_2ano if col != "id"
    ])

    # --- Busca de dados da tabela do 1° ano ---
    cursor_1ano.execute(f"SELECT {colunas_1ano_str} FROM {tabela_1ano};")
    dados_1ano = cursor_1ano.fetchall()

    

    # --- Faz o merge na tabela de destino, se nao houver conflitos no id irá adicionar o que veio do banco, porém caso tenha o id, atualiza se necessário ---
    try:
        for linha in dados_1ano:
            query = f"""
                INSERT INTO {tabela_2ano} ({colunas_2ano_str})
                VALUES ({placeholders})
                ON CONFLICT (id) DO UPDATE
                SET {excluded}
                WHERE {condicoes};
            """
            cursor_2ano.execute(query, linha)
        conn_2ano.commit()
        print(f"Dados da tabela {tabela_1ano} inseridos/atualizados com sucesso na tabela {tabela_2ano}.")
    except Exception as e:
        conn_2ano.rollback()
        print("Erro ao inserir/atualizar dados: ",e)



***Teste Para Iteração Entre Duas Tabelas***

In [66]:
# iteração para executar o RPA em todas as tabelas definidas -- exemplo de como funciona abaixo
for (key_1, valor_1), (key_2, valor_2) in zip(tabelas_1ano.items(), tabelas_2ano.items()):
    
    print(f"Tabela 1: {key_1} -> Colunas: {valor_1}")
    print(f"Tabela 2: {key_2} -> Colunas: {valor_2}")


Tabela 1: plano -> Colunas: ['id', 'nome', 'preco', 'armazenamento']
Tabela 2: planos -> Colunas: ['id', 'nome', 'preco', 'armazenamento']
Tabela 1: condena -> Colunas: ['id', 'nome', 'tipo']
Tabela 2: condena -> Colunas: ['id', 'nome', 'tipo']
Tabela 1: empresa -> Colunas: ['id', 'nome', 'cnpj']
Tabela 2: empresa -> Colunas: ['id', 'nome']


***Execução do RPA***

In [ ]:

# Executa o RPA para cada tabela
for (key_1, valor_1), (key_2, valor_2) in zip(tabelas_1ano.items(), tabelas_2ano.items()):
    if key_1 == "empresa":
        valor_1.remove("cnpj")  # Remover a coluna 'cnpj' da tabela do 1° ano, pois não existe na tabela do 2° ano e não é utilizado
    RPA_inter(key_1, valor_1, key_2, valor_2, conn_1ano, conn_2ano)

